In [1]:
import pandas as pd
import altair as alt
import warnings
import numpy as np
import requests
import datetime
warnings.filterwarnings("ignore")
alt.renderers.set_embed_options(theme='dark')
pd.set_option("display.max_colwidth", 400)
pd.set_option("display.max_rows", 400)


In [5]:
class PrismVestedDataProvider:
    def __init__(self, claim, path='../data'):
        self.path = path
        self.claim = claim
        self.prism_claim = '3622a25b-bce9-4d69-8153-3681d2fd1c6a'
        pass
        
    def load(self):
        self.actions = {
                'Prism':'Claim',
                'xPrism': 'Claim and Stake',
                'AMPS':'Claim and Stake and Pledge'
            }
        if(self.prism_claim):
            self.prism_claim_df = self.claim(self.prism_claim)
        if(len(self.prism_claim_df.columns) == 0):
            self.prism_claim_df = pd.DataFrame([
                    ['2022-03-11 17:45:27.509','terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs5vs',
                     'Prism',2000000000,'4E8269A29F8FCA39742D30171DDEF4F70D470553521A268A7BAA150E80FED333'],
                    ['2022-03-11 17:45:27.509','terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs3vs',
                     'xPrism',3000000000,'4E8269A29F8FCA39742D30171DDEF4F70D470553521A268A7BAA150E80FED333'],
                    ['2022-03-11 17:45:27.509','terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs4vs',
                     'xPrism',4000000000,'4E8269A29F8FCA39742D30171DDEF4F70D470553521A268A7BAA150E80FED333'],
                    ['2022-03-12 17:45:27.509','terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs5vs',
                     'AMPS',2000000000,'4E8269A29F8FCA39742D30171DDEF4F70D470553521A268A7BAA150E80FED333'],
                    ['2022-03-12 17:45:27.509','terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs5vs',
                     'Prism',1000000000,'4E8269A29F8FCA39742D30171DDEF4F70D470553521A268A7BAA150E80FED333'],
                    ['2022-03-12 17:45:27.509','terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs5vs',
                     'AMPS',500000000,'4E8269A29F8FCA39742D30171DDEF4F70D470553521A268A7BAA150E80FED333'],
                    ['2022-03-12 17:45:27.509','terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs6vs',
                     'xPrism',12000000000,'4E8269A29F8FCA39742D30171DDEF4F70D470553521A268A7BAA150E80FED333'],
                ], columns=['block_timestamp','user','action','amount','tx_id'])
        
    def parse(self):
        df = self.prism_claim_df
        df.amount = df.amount/1000000
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        df['hr'] = df.block_timestamp.str[:-5] + '00:00.000'
        df['day'] = df.block_timestamp.str[:-9]
        df.action = df.action.map(self.actions)
        self.prism_claim_df = df
        

In [6]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [7]:
dp = PrismVestedDataProvider(claim)
dp.load()
dp.parse()

In [85]:
df = dp.prism_claim_df.groupby('action').amount.sum().reset_index()
df.columns = ['Claim Action','Amount of PRISM']
chart = alt.Chart(df).mark_arc(innerRadius=60).encode(
    theta=alt.Theta(field="Amount of PRISM", type="quantitative"),
    color=alt.Color(field="Claim Action", type="nominal",
            #sort=['MARS & UST','MARS','UST'],
            scale=alt.Scale(scheme='lightorange'),
            legend=alt.Legend(
            orient='none',
            padding=10,
            legendY=-10,
            direction='vertical')),
    tooltip=['Claim Action','Amount of PRISM']
).configure_view(strokeOpacity=0)
chart

alt.Chart(...)

In [86]:
df = dp.prism_claim_df.groupby(['action','day']).amount.sum().reset_index()
n_data = 20
if df.day.nunique() < n_data:
    extra_data = []
    for i in range(n_data-df.day.nunique()):
        extra_data.append(['Claim',(pd.to_datetime(df.day.max())+datetime.timedelta(days=i)).strftime("%Y-%m-%d"),0])
    df2 = df.append(pd.DataFrame(extra_data, columns=df.columns))
else:
    df2 = df
chart = alt.Chart(df2).mark_bar().encode(
    x=alt.X('day:T', sort=alt.EncodingSortField(order='ascending')),
    y="amount:Q",
    color=alt.Color('action', 
                    scale=alt.Scale(scheme='lightorange'),
                    legend=alt.Legend(
                            orient='top-right',
                            padding=5,
                            legendY=0,
                            direction='vertical'))
    ,tooltip=[alt.Tooltip('day:T', format='%Y-%m-%d %H:%M'), 'amount']
).properties(width=700).configure_axisX(
    labelAngle=0
).configure_view(strokeOpacity=0)
chart

alt.Chart(...)

In [87]:
df = dp.prism_claim_df.groupby(['action','day']).user.nunique().reset_index()
n_data = 20
if df.day.nunique() < n_data:
    extra_data = []
    for i in range(n_data-df.day.nunique()):
        extra_data.append(['Claim',(pd.to_datetime(df.day.max())+datetime.timedelta(days=i)).strftime("%Y-%m-%d"),0])
    df2 = df.append(pd.DataFrame(extra_data, columns=df.columns))
else:
    df2 = df
chart = alt.Chart(df2).mark_bar().encode(
    x=alt.X('day:T', sort=alt.EncodingSortField(order='ascending')),
    y="user:Q",
    color=alt.Color('action', 
                    scale=alt.Scale(domain=['Claim','Claim and Stake','Claim and Stake and Pledge'], range=['#c7208c','#fbb7bd','#ffffff']),
                    legend=alt.Legend(
                            orient='top-right',
                            padding=5,
                            legendY=0,
                            direction='vertical'))
    ,tooltip=[alt.Tooltip('day:T', format='%Y-%m-%d %H:%M'), 'user']
).properties(width=700).configure_axisX(
    labelAngle=0
).configure_view(strokeOpacity=0)
chart

alt.Chart(...)

In [71]:
class AMPSChart:
    
    def __init__(self):
        self.cols_dict = {
            'user_xprism': 'Amount of xPRISM ',
            'user_xprism_label': 'Amount of xPRISM',
            'boost_accrual_start_time_days_int': 'Number of days pledged for',
            'index': 'Number of users',
            'boost_accrual_start_time_days': 'Current number of days pledged for',
            'user_yluna': 'Amount of yLUNA',
            'addr': 'User address',
            'current_daily_rewards': 'Current daily rewards (PRISM)',
            'n_addr': 'Number of users',
            'boost_apr': 'Boost APR (%)'
        }
    def current_daily_rewards_cat(self, x):
        if x<1:
            return '1'
        if x < 100:
            v = (int(x/10)+1)*10
            return f'{v-9}-{v}'
        if x < 1000:
            v = (int(x/100)+1)*100
            return f'{v-99}-{v}'
        if x>=1000:
            return '> 1000'

    def time_xprism_yluna(self, df):
        cols_dict = self.cols_dict
        df['url'] = 'https://finder.extraterrestrial.money/mainnet/address/'+df['addr']
        if(len(df)>5000):
            df = df.sample(n=5000, random_state=1)
        else:
            df = df
        df2 = df.rename(columns=cols_dict)
        chart =alt.Chart(df2).mark_point(opacity=1, filled=True).encode(
            y=alt.Y(cols_dict['user_xprism']+":Q"),
            x=alt.X(cols_dict['boost_accrual_start_time_days']+":Q"),
            href='url:N',
            color=alt.Color(cols_dict['user_yluna'],
                scale=alt.Scale(scheme='redpurple', domain=[0,df2[cols_dict['user_yluna']].max()/5]),
                legend=alt.Legend(
                            orient='top-left',
                            padding=0,
                            legendY=0,
                            direction='vertical')),
            tooltip=[cols_dict['addr'],cols_dict['user_xprism'],cols_dict['user_yluna'],
                     cols_dict['boost_accrual_start_time_days']]
        ).configure_view(strokeOpacity=0).interactive()
        return chart.properties(width=600)
    
    def users_daily_rewards(self, df):
        cols_dict = self.cols_dict
        s = ['1']
        for i in range(1,11):
            v = i*10
            s.append(f'{v-9}-{v}')
        for i in range(2,11):
            v = i*100
            s.append(f'{v-99}-{v}')
        s.append('>1000')
        df2 = df.copy()
        df2['current_daily_rewards'] = df2['current_daily_rewards'].apply(lambda x: 1 if int(x/10)*10 == 0 else int(x/10)*10)
        df2['current_daily_rewards'] = df2.current_daily_rewards.apply(self.current_daily_rewards_cat)
        #print(df2.current_daily_rewards.value_counts())
        df2['n_addr'] = df2['addr']
        df2 = df2.groupby('current_daily_rewards').n_addr.count().reset_index()
        df2 = df2.rename(columns=cols_dict)
        chart = alt.Chart(df2).mark_bar().encode(
            x=alt.X(cols_dict['current_daily_rewards']+':N', \
                    sort=s,\
                    axis=alt.Axis(tickCount=10, labelAngle=-30, tickBand = 'center')),
            y=cols_dict['n_addr']+':Q',
            tooltip=[cols_dict['current_daily_rewards'],cols_dict['n_addr']+':Q']
        ).configure_mark(
            color='#DAFD91'
        ).configure_view(strokeOpacity=0)
        return chart.properties(width=600)
    
    def users_boost_apr(self, df):
        cols_dict = self.cols_dict
        df2 = df.copy().fillna(0)
        df2['boost_apr'] = df2['boost_apr'].apply(lambda x: 0 if int(x/10)*10 == 0 else int(x/10)*10)
        df2 = df2[df2['boost_apr']>0]
        df2['n_addr'] = df2['addr']
        df2 = df2.groupby('boost_apr').n_addr.count().reset_index()
        df2 = df2.rename(columns=cols_dict)
        chart = alt.Chart(df2).mark_bar().encode(
            x=alt.X(cols_dict['boost_apr']+':N', \
                    axis=alt.Axis(tickCount=10, labelAngle=-90, tickBand = 'center')),
            y=cols_dict['n_addr']+':Q',
            tooltip=[cols_dict['boost_apr'],cols_dict['n_addr']+':Q']
        ).configure_mark(
            color='#ccf4ed'
        ).configure_view(strokeOpacity=0)
        return chart.properties(width=600)
    
    def users_days_pledged(self,df):
        cols_dict = self.cols_dict
        df2 = df.boost_accrual_start_time_days.apply(int).value_counts().reset_index()
        df2.columns = ['Current number of days pledged for','Number of users']
        chart = alt.Chart(df2).mark_bar().encode(
            x=alt.X('Current number of days pledged for'+':N', \
                    scale=alt.Scale(domain=list(range(df2['Current number of days pledged for'].max()+1))),\
                    axis=alt.Axis(tickCount=10, labelAngle=0, tickBand = 'center')),
            y='Number of users'+":Q",
            tooltip=['Current number of days pledged for','Number of users'+":Q"]
        ).configure_mark(
            color='#DAFD91'
        ).configure_view(strokeOpacity=0)
        return chart.properties(width=600)
    
    def xprisms_days_pledged(self,df):
        cols_dict = self.cols_dict
        df2 = df.groupby('boost_accrual_start_time_days_int').user_xprism.sum().reset_index()
        df2 = df2.rename(columns=cols_dict)
        df2[cols_dict['user_xprism_label']] = df2[cols_dict['user_xprism']].apply(lambda x: str(round(x/1000000,2))+'M')
        chart = alt.Chart(df2).mark_bar().encode(
            x=alt.X(cols_dict['boost_accrual_start_time_days_int']+':N', \
                    scale=alt.Scale(domain=list(range(df2[cols_dict['boost_accrual_start_time_days_int']].max()+1))),\
                    axis=alt.Axis(tickCount=10, labelAngle=0, tickBand = 'center')),
            y=cols_dict['user_xprism']+':Q',
            tooltip=[cols_dict['boost_accrual_start_time_days_int'],cols_dict['user_xprism_label']+':N']
        ).configure_mark(
            color='#ccf4ed'
        ).configure_view(strokeOpacity=0)
        return chart.properties(width=600)
    
    def pledge_unpledge_daily(self,amps_activity_df):
        pl = amps_activity_df[['day','pledge_amount']]
        pl.columns = ['Day','Amount of xPRISM']
        pl['Action'] = 'Pledge'
        unpl = amps_activity_df[['day','unpledge_amount']]
        unpl.columns = ['Day','Amount of xPRISM']
        unpl['Amount of xPRISM'] = -unpl['Amount of xPRISM']
        unpl['Action'] = 'Unpledge'
        df = pl.append(unpl)
        df['Amount of xPRISM'] = df['Amount of xPRISM'].apply(lambda: round(x,2))
        return alt.Chart(df).mark_line(point=True).encode(
             x=alt.X('Day:T', sort=alt.EncodingSortField(order='ascending')),
             y=alt.Y("Amount of xPRISM:Q",scale=alt.Scale(domain=[-3000000,18000000])),
            color=alt.Color('Action:N', 
                            scale=alt.Scale(scheme='set2'),
                            legend=alt.Legend(
                                    orient='top-left',
                                    padding=5,
                                    legendY=0,
                                    direction='vertical')),
            tooltip=[alt.Tooltip('Day:T', format='%Y-%m-%d'), 'Action', 'Amount of xPRISM']
        ).properties(width=700).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0)
        
    def number_users_pledging(self,amps_activity_df):
        pl = amps_activity_df[['day','n_bonding_users']]
        pl.columns = ['Day','Number of users']
        pl['Action'] = 'Pledge'
        unpl = amps_activity_df[['day','n_unbonding_users']]
        unpl.columns = ['Day','Number of users']
        unpl['Action'] = 'Unpledge'
        df = pl.append(unpl)
        return alt.Chart(df).mark_bar().encode(
             x=alt.X('Day:T', sort=alt.EncodingSortField(order='ascending')),
             y=alt.Y("Number of users:Q"),
            color=alt.Color('Action:N', 
                    scale=alt.Scale(scheme='set2'),
                    legend=alt.Legend(
                            orient='top-left',
                            padding=5,
                            legendY=0,
                            direction='vertical')),
           tooltip=[alt.Tooltip('Day:T', format='%Y-%m-%d'), 'Action', 'Number of users']
        ).configure_mark(
            color='#DAFD91'
        ).properties(width=700).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0)

## Query

In [92]:
cp = AMPSChart()

In [97]:
cp.xprisms_days_pledged(dp.amps)

alt.Chart(...)